# Parse Input

In [1]:

with open("day5_input_rules.txt") as input_rules, open("day5_input_pages.txt") as input_updates: 
    str_rules = list(input_rules)
    # str_updates = list([int(x) for x in "".join([l.replace("\n", ",") for l in input_pages]).split(",")])
    updates = list([int(p) for p in update.split(",")] for update in input_updates)
str_rules, updates

(['59|55\n',
  '48|63\n',
  '48|74\n',
  '26|77\n',
  '26|54\n',
  '26|99\n',
  '63|43\n',
  '63|65\n',
  '63|11\n',
  '63|42\n',
  '91|64\n',
  '91|77\n',
  '91|71\n',
  '91|42\n',
  '91|33\n',
  '46|63\n',
  '46|48\n',
  '46|72\n',
  '46|21\n',
  '46|12\n',
  '46|82\n',
  '21|32\n',
  '21|11\n',
  '21|52\n',
  '21|64\n',
  '21|59\n',
  '21|34\n',
  '21|33\n',
  '99|82\n',
  '99|37\n',
  '99|21\n',
  '99|27\n',
  '99|92\n',
  '99|79\n',
  '99|56\n',
  '99|12\n',
  '38|51\n',
  '38|47\n',
  '38|77\n',
  '38|88\n',
  '38|34\n',
  '38|52\n',
  '38|32\n',
  '38|11\n',
  '38|64\n',
  '33|42\n',
  '33|54\n',
  '33|56\n',
  '33|47\n',
  '33|93\n',
  '33|31\n',
  '33|52\n',
  '33|99\n',
  '33|34\n',
  '33|77\n',
  '32|52\n',
  '32|88\n',
  '32|56\n',
  '32|45\n',
  '32|77\n',
  '32|18\n',
  '32|42\n',
  '32|46\n',
  '32|71\n',
  '32|47\n',
  '32|33\n',
  '96|45\n',
  '96|31\n',
  '96|11\n',
  '96|74\n',
  '96|59\n',
  '96|55\n',
  '96|42\n',
  '96|71\n',
  '96|39\n',
  '96|43\n',
  '96|44\n',

In [ ]:
# Part 1

with open("day5_input_rules.txt") as input_rules, open("day5_input_pages.txt") as input_updates: 
    str_rules = list(input_rules)
    # str_updates = list([int(x) for x in "".join([l.replace("\n", ",") for l in input_pages]).split(",")])
    updates = list([int(p) for p in update.split(",")] for update in input_updates)

class rule:
    before: int
    after: int

    def __init__(self, str_rule):
        self.before, self.after = [int(p) for p in str_rule.split('|')]

    def __repr__(self):
        return f"{self.before} | {self.after}"
    
class number:
    number: int
    goes_before: set[int]
    goes_after: set[int]

    def copy_and_restrict(self, update):
        update = set(update)
        return number(self.number, self.goes_before & update, self.goes_after & update )

    def __init__(self, number, goes_before, goes_after):
        self.number = number
        self.goes_before = goes_before
        self.goes_after = goes_after

    def __repr__(self):
        before_str = ", ".join([str(x) for x in self.goes_before])
        after_str = ", ".join([str(x) for x in self.goes_after])
        return f"Number: {self.number}\nGoes Before: {before_str}\nGoes After: {after_str}\n\n"
    
    
rules = [rule(r) for r in str_rules]
int_numbers = {x.before for x in rules} | {x.after for x in rules}

numbers = {}
for n in int_numbers:
    goes_before = {r.after for r in rules if r.before == n}
    goes_after = {r.before for r in rules if r.after == n}
    numbers[n] = number(n, goes_before, goes_after)

update_outputs = []
for update in updates:
    is_valid = False 
    update_output = []
    ignored_pages_before = set()
    ignored_pages_after = set()
    pages = [numbers[n].copy_and_restrict(update) for n in update] 
    while(not is_valid):
        for page in [p for p in pages if p.number not in ignored_pages_before | ignored_pages_after]:
            # print(f"{page.number} - \npage.goes_before: {page.goes_before} \npage.goes_after: {page.goes_after}\n\n")

            if not (page.goes_before - ignored_pages_before):
                # Go to middle 
                update_output.insert(len(ignored_pages_after), page.number)
                ignored_pages_before.add(page.number)
                break

            if not (page.goes_after - ignored_pages_after):
                # Go to middle 
                update_output.insert(len(ignored_pages_after), page.number)
                ignored_pages_after.add(page.number)
                break
        if len(update_output) == len(update):
            is_valid = True
            update_outputs.append(update_output)

# Validate
for update_output in update_outputs:
    for i, page in enumerate(update_output):
        if sum([x for x in numbers[page].goes_before if x in update_output[:i]]):
            print("FAILURE BEFORE")
        if sum([x for x in numbers[page].goes_after if x in update_output[i:]]):
            print("FAILURE AFTER")

output_number = sum([u[int((len(u) - 1)/2)] for u in update_outputs])
update_outputs, output_number